# 정적인 웹사이트 크롤링
_본 자료는 안수찬 강사님의 파이썬을 활용한 업무자동화 Camp (fast campus)의 강의자료를 기반으로 만들어졌습니다._  
만든이 : 김보섭  

## Preliminary
* 웹 사이트 (웹 클라이언트) : HTML, CSS, Javascript
* HTML : 사이트의 구조
* CSS : 사이트의 스타일
* Javascript : 사이트의 동적인 기능들 (애니메이션, 데이터를 서버에서 불러오는 기능)
* 데이터를 우리에게 뿌려주는 주체
    * 웹 서버 => HTML (Server rendering)
    * 웹 클라이언트 (Javascript) => HTML (Client Rendering)

## Summary
* 정적인 웹사이트
    * HTML 파일을 다운로드 (Crawling, Scraping)
    * 우리가 원하는 데이터의 위치를 찾아서 추출! (Parsing)

### 네이버 실시간 검색어 데이터
'realrank'라는 id를 가진 ol이라고 하는 태그 내에 10개의 li 태그가 있고, 그 li 태그안에 있는 텍스트를 뽑아내자.  

* CSS Selector (CSS 선택자)
    * ol#realrank li
    * id = '#', class ='.'

#### GET으로 요청 

In [1]:
# 1. html
import requests
response = requests.get('http://www.naver.com/')
type(response.text)
# response.text (string)

str

#### BeautifulSoup으로 parsing

In [2]:
#### 2. parsing
from bs4 import BeautifulSoup
bs = BeautifulSoup(response.text, 'html.parser')
type(bs)
# bs.. 우리가 쉽게 파싱할 수 있도록 HTML을 구조화해둔 객체

bs4.BeautifulSoup

#### 실시간 검색어 키워드

In [3]:
elements = bs.select('ol#realrank li')

In [4]:
element = elements[0]

In [5]:
type(element)

bs4.element.Tag

In [6]:
element

<li class="up" value="1"><a href="http://search.naver.com/search.naver?where=nexearch&amp;query=%EA%B3%B5%EA%B0%81+%EA%B8%B0%EB%8F%99%EB%8C%80&amp;sm=top_lve&amp;ie=utf8" title=""><span class="ell">공각 기동대</span><span class="tx">상승</span><span class="ic"></span><span class="rk">93</span></a></li>

In [7]:
element.select_one('span.ell').text

'공각 기동대'

In [8]:
# 주의할점 select_one으로 찾으면 그 자체로 bs4.element.Tag 이지만
# select는 해당 CSS에 걸리는 여러 개를 찾는 것이므로 결과물이 bs4.element.Tag가 List의 하나하나의 값으로 들어가있음
print(element.select_one('span.ell'), element.select('span.ell'))

<span class="ell">공각 기동대</span> [<span class="ell">공각 기동대</span>]


#### 실시간 검색어 링크

In [9]:
element

<li class="up" value="1"><a href="http://search.naver.com/search.naver?where=nexearch&amp;query=%EA%B3%B5%EA%B0%81+%EA%B8%B0%EB%8F%99%EB%8C%80&amp;sm=top_lve&amp;ie=utf8" title=""><span class="ell">공각 기동대</span><span class="tx">상승</span><span class="ic"></span><span class="rk">93</span></a></li>

In [10]:
element.select_one('a')

<a href="http://search.naver.com/search.naver?where=nexearch&amp;query=%EA%B3%B5%EA%B0%81+%EA%B8%B0%EB%8F%99%EB%8C%80&amp;sm=top_lve&amp;ie=utf8" title=""><span class="ell">공각 기동대</span><span class="tx">상승</span><span class="ic"></span><span class="rk">93</span></a>

In [11]:
print(element.select_one('a').attrs)
print(type(element.select_one('a').attrs)) # dictionary의 형태

{'href': 'http://search.naver.com/search.naver?where=nexearch&query=%EA%B3%B5%EA%B0%81+%EA%B8%B0%EB%8F%99%EB%8C%80&sm=top_lve&ie=utf8', 'title': ''}
<class 'dict'>


In [12]:
element.select_one('a').attrs.get('href')

'http://search.naver.com/search.naver?where=nexearch&query=%EA%B3%B5%EA%B0%81+%EA%B8%B0%EB%8F%99%EB%8C%80&sm=top_lve&ie=utf8'

In [13]:
# 주의할점 select_one으로 찾으면 그 자체로 bs4.element.Tag 이지만
# select는 해당 CSS에 걸리는 여러 개를 찾는 것이므로 결과물이 bs4.element.Tag가 List의 하나하나의 값으로 들어가있음
print(element.select_one('a'))
print(element.select('a'))

<a href="http://search.naver.com/search.naver?where=nexearch&amp;query=%EA%B3%B5%EA%B0%81+%EA%B8%B0%EB%8F%99%EB%8C%80&amp;sm=top_lve&amp;ie=utf8" title=""><span class="ell">공각 기동대</span><span class="tx">상승</span><span class="ic"></span><span class="rk">93</span></a>
[<a href="http://search.naver.com/search.naver?where=nexearch&amp;query=%EA%B3%B5%EA%B0%81+%EA%B8%B0%EB%8F%99%EB%8C%80&amp;sm=top_lve&amp;ie=utf8" title=""><span class="ell">공각 기동대</span><span class="tx">상승</span><span class="ic"></span><span class="rk">93</span></a>]


#### 실시간 검색어, 해당 검색어의 링크를 parsing

In [14]:
# List of dictionary의 형태로
data = [{'keyword' : tmp.select_one('span.ell').text,
  'address' : tmp.select_one('a').attrs.get('href')} for tmp in bs.select('ol#realrank li')]

In [15]:
print(data[0:4])

[{'keyword': '공각 기동대', 'address': 'http://search.naver.com/search.naver?where=nexearch&query=%EA%B3%B5%EA%B0%81+%EA%B8%B0%EB%8F%99%EB%8C%80&sm=top_lve&ie=utf8'}, {'keyword': '엄석대', 'address': 'http://search.naver.com/search.naver?where=nexearch&query=%EC%97%84%EC%84%9D%EB%8C%80&sm=top_lve&ie=utf8'}, {'keyword': '스칼렛요한슨', 'address': 'http://search.naver.com/search.naver?where=nexearch&query=%EC%8A%A4%EC%B9%BC%EB%A0%9B%EC%9A%94%ED%95%9C%EC%8A%A8&sm=top_lve&ie=utf8'}, {'keyword': '설민석', 'address': 'http://search.naver.com/search.naver?where=nexearch&query=%EC%84%A4%EB%AF%BC%EC%84%9D&sm=top_lve&ie=utf8'}]


In [16]:
import pandas as pd
pd.DataFrame(data)

,address,keyword
0,http://search.naver.com/search.naver?where=nex...,공각 기동대
1,http://search.naver.com/search.naver?where=nex...,엄석대
2,http://search.naver.com/search.naver?where=nex...,스칼렛요한슨
3,http://search.naver.com/search.naver?where=nex...,설민석
4,http://search.naver.com/search.naver?where=nex...,고등래퍼
5,http://search.naver.com/search.naver?where=nex...,인터파크티켓
6,http://search.naver.com/search.naver?where=nex...,영화순위
7,http://search.naver.com/search.naver?where=nex...,bgf리테일 채용
8,http://search.naver.com/search.naver?where=nex...,나혼자산다
9,http://search.naver.com/search.naver?where=nex...,양홍원


### 네이버 블로그 포스트 가져오기

In [17]:
response = requests.get('https://search.naver.com/search.naver?where=post&sm=tab_pge&query=%ED%8C%8C%EC%9D%B4%EC%8D%AC&st=sim&date_option=0&date_from=&date_to=&dup_remove=1&post_blogurl=&post_blogurl_without=&srchby=all&nso=&ie=utf8&start=1')
bs = BeautifulSoup(response.text, 'html.parser')

In [18]:
contents = bs.select('ul#elThumbnailResultArea li.sh_blog_top dt')

#### 블로그 포스트 제목

In [19]:
contents[0:3]

[<dt><a class="sh_blog_title _sp_each_url _sp_each_title" href="http://rmsep39.tistory.com/1905" onclick="return goOtherCR(this, 'a=blg*t.tit&amp;r=2&amp;i=a00000fa_24235b281218b73aca74f1c5&amp;u='+urlencode(this.href))" target="_blank" title="파이썬프로그래밍 기업에서 선호하는 이유"><strong class="hl">파이썬</strong>프로그래밍 기업에서 선호하는 이유</a></dt>,
 <dt><a class="sh_blog_title _sp_each_url _sp_each_title" href="http://blog.naver.com/infopub?Redirect=Log&amp;logNo=220945501558" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=2&amp;i=90000003_00000000000000337160CD76&amp;u='+urlencode(this.href))" target="_blank" title="초보자를 위한 파이썬 200제">초보자를 위한 <strong class="hl">파이썬</strong> 200제</a></dt>,
 <dt><a class="sh_blog_title _sp_each_url _sp_each_title" href="http://1984.tistory.com/448" onclick="return goOtherCR(this, 'a=blg*t.tit&amp;r=2&amp;i=a00000fa_127472e44f8c39b566a8fa88&amp;u='+urlencode(this.href))" target="_blank" title="파이썬 프로그래밍 배워야하는 3가지이유!"><strong class="hl">파이썬</strong> 프로그래밍 배워야하는 3가지이유!</a></dt

In [20]:
contents[0]

<dt><a class="sh_blog_title _sp_each_url _sp_each_title" href="http://rmsep39.tistory.com/1905" onclick="return goOtherCR(this, 'a=blg*t.tit&amp;r=2&amp;i=a00000fa_24235b281218b73aca74f1c5&amp;u='+urlencode(this.href))" target="_blank" title="파이썬프로그래밍 기업에서 선호하는 이유"><strong class="hl">파이썬</strong>프로그래밍 기업에서 선호하는 이유</a></dt>

In [21]:
contents[0].select_one('a').attrs.get('title')

'파이썬프로그래밍 기업에서 선호하는 이유'

#### 블로그 포스트 주소 

In [22]:
contents[0:3]

[<dt><a class="sh_blog_title _sp_each_url _sp_each_title" href="http://rmsep39.tistory.com/1905" onclick="return goOtherCR(this, 'a=blg*t.tit&amp;r=2&amp;i=a00000fa_24235b281218b73aca74f1c5&amp;u='+urlencode(this.href))" target="_blank" title="파이썬프로그래밍 기업에서 선호하는 이유"><strong class="hl">파이썬</strong>프로그래밍 기업에서 선호하는 이유</a></dt>,
 <dt><a class="sh_blog_title _sp_each_url _sp_each_title" href="http://blog.naver.com/infopub?Redirect=Log&amp;logNo=220945501558" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=2&amp;i=90000003_00000000000000337160CD76&amp;u='+urlencode(this.href))" target="_blank" title="초보자를 위한 파이썬 200제">초보자를 위한 <strong class="hl">파이썬</strong> 200제</a></dt>,
 <dt><a class="sh_blog_title _sp_each_url _sp_each_title" href="http://1984.tistory.com/448" onclick="return goOtherCR(this, 'a=blg*t.tit&amp;r=2&amp;i=a00000fa_127472e44f8c39b566a8fa88&amp;u='+urlencode(this.href))" target="_blank" title="파이썬 프로그래밍 배워야하는 3가지이유!"><strong class="hl">파이썬</strong> 프로그래밍 배워야하는 3가지이유!</a></dt

In [23]:
contents[0].select_one('a').attrs.get('href')

'http://rmsep39.tistory.com/1905'

#### 블로그 포스트의 제목과, 블로그 포스트 주소 parsing 

In [24]:
data = [
    {'title': content.select_one('a').attrs.get('title'),
    'address' : content.select_one('a').attrs.get('href')}
    for content
    in contents
]

In [25]:
len(data)

10

In [26]:
data

[{'address': 'http://rmsep39.tistory.com/1905',
  'title': '파이썬프로그래밍 기업에서 선호하는 이유'},
 {'address': 'http://blog.naver.com/infopub?Redirect=Log&logNo=220945501558',
  'title': '초보자를 위한 파이썬 200제'},
 {'address': 'http://1984.tistory.com/448', 'title': '파이썬 프로그래밍 배워야하는 3가지이유!'},
 {'address': 'http://chogar.blog.me/220942149662',
  'title': '마인크래프트를 활용한 파이썬 프로그래밍 과정 시작!'},
 {'address': 'http://edujoa.tistory.com/1389', 'title': '프로그래밍 입문 파이썬부터 시작!!'},
 {'address': 'http://blog.naver.com/nasu0210?Redirect=Log&logNo=220932224509',
  'title': '대구 프로그래밍 학원 파이썬 1개월 완성 주말 과정 개설'},
 {'address': 'http://blog.alyac.co.kr/985',
  'title': '패치되지 않은 파이썬과 자바 취약점, 해커들이 FTP 인젝션을 통해 방화벽 우회하도록 허용해'},
 {'address': 'http://shaeod.tistory.com/949',
  'title': '[개발] 파이썬 다운로드 및 윈도우에 설치하는 방법'},
 {'address': 'http://rmsep39.tistory.com/2056',
  'title': '파이썬 강좌, 비전공자도 배울 수 있다!'},
 {'address': 'http://blog.fastcampus.co.kr/220942981452',
  'title': '파이썬과 장고의 실무 개발 노하우를 전수해드리겠습니다.'}]

In [27]:
pd.DataFrame(data)

,address,title
0,http://rmsep39.tistory.com/1905,파이썬프로그래밍 기업에서 선호하는 이유
1,http://blog.naver.com/infopub?Redirect=Log&log...,초보자를 위한 파이썬 200제
2,http://1984.tistory.com/448,파이썬 프로그래밍 배워야하는 3가지이유!
3,http://chogar.blog.me/220942149662,마인크래프트를 활용한 파이썬 프로그래밍 과정 시작!
4,http://edujoa.tistory.com/1389,프로그래밍 입문 파이썬부터 시작!!
5,http://blog.naver.com/nasu0210?Redirect=Log&lo...,대구 프로그래밍 학원 파이썬 1개월 완성 주말 과정 개설
6,http://blog.alyac.co.kr/985,"패치되지 않은 파이썬과 자바 취약점, 해커들이 FTP 인젝션을 통해 방화벽 우회하도..."
7,http://shaeod.tistory.com/949,[개발] 파이썬 다운로드 및 윈도우에 설치하는 방법
8,http://rmsep39.tistory.com/2056,"파이썬 강좌, 비전공자도 배울 수 있다!"
9,http://blog.fastcampus.co.kr/220942981452,파이썬과 장고의 실무 개발 노하우를 전수해드리겠습니다.
